# Lyman-$\alpha$ Absorption Lines

Figure 2.6 from Chapter 2 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Plot of Lyman-$\alpha$ absorption lines with central optical depths $\tau_0$ of 1 through 1000 and
$b=10\,km\,s^{-1}$. We use a numerical implementation of the Voigt function as the real part of the 
[Faddeeva function](https://en.wikipedia.org/wiki/Faddeeva_function) $w(z)$ in the SciPy specials package, 
function wofz().

Optical depths plotted are $\tau_0$={1,2,10,100,1000}.

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter, FormatStrFormatter

from scipy import special

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
figName = 'Fig2_6' 

# graphic aspect ratio = width/height

aspect = 4.0/3.0 # 4:3

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## Voigt-Hjerting Function

Formal implementation using Faddeeva or Kramp function w(z)
implemented in scipy.special as wofz(). plucked off SciPy-User
from archives via Google.  For a detailed derivation, the
Wikipedia articles are good resources:
 * [Voigt Profile](http://en.wikipedia.org/wiki/Voigt_profile)
 * [Faddeeva Function](http://en.wikipedia.org/wiki/Faddeeva_function)

The Voight function is the real part of the Faddeeva function.  We're slightly finessing the normalization
here for illustration purposes.

The Lyman-$\alpha$ line will be computed for the following properties:
 * Optical depth at line center of $\tau_0$={1,2,10,100,1000}
 * Doppler parameter $b=10\,km\,s^{-1}$.
 * Lorentzian natural width for HI Lyman $\alpha$ of $\gamma=2.458\times10^{-05}$ Angstroms
 * Line center $\lambda_0=1215.67$ Angstroms

In [ ]:
def H(a,u):
    return special.wofz(u + 1j*a).real

# Constants

c = 2.99792458e05  # speed of light in km/sec (CODATA 2018)

# Properties of HI Lyman-alpha, mix of constants and parameters

lam0 = 1215.67    # Lyman-alpha in Angstroms
b = 10.0          # Doppler width in km/sec
blam = lam0*b/c   # Doppler width in Angstroms
gamma = 2.458e-05 # Natural width for HI Lyman Alpha in Angstroms
a = gamma/blam    # line a factor

I0 = 1.0          # default central intensity

# Range of optical depths

tau0 = [1.0,2.0,10.0,100.0,1000.0]
xTau = [lam0,lam0,1215.705,1215.735,1215.78]
yTau = [0.43,0.15,0.02,0.02,0.02]
# plotting limits

dLam = 0.001 # pixel size in Angstroms
wPix = 300   # calculation width +/-pixels 

minLam = lam0 - dLam*wPix
maxLam = lam0 + dLam*wPix
numPix = 2*wPix+1

# wavelengths and u

lam = np.linspace(minLam,maxLam,num=numPix)
u = (lam-lam0)/blam

lMin = 1215.47
lMax = 1215.87
yMin = -0.05
yMax = 1.05

### Make the Plot

All lines are solid black and labeled.

In [ ]:
fig,ax = plt.subplots()

fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

plt.xlim(lMin,lMax)
ax.xaxis.set_major_locator(MultipleLocator(0.1))
ax.xaxis.set_minor_locator(MultipleLocator(0.01))
ax.xaxis.set_major_formatter(FormatStrFormatter('%6.1f'))
plt.xlabel(r'Wavelength [\AA]',fontsize=axisFontSize)

plt.ylim(yMin,yMax)
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
plt.ylabel(r'Intensity',fontsize=axisFontSize)

# plot the curves

for i, t0 in enumerate(tau0):
    plt.plot(lam,I0*np.exp(-t0*H(a,u)),'-',color='black',lw=1,zorder=10)
    if t0 == 1.0:
        label = r'$\tau_0=1$'
    else:
        label = f'{int(t0):d}'
        
    plt.text(xTau[i],yTau[i],label,color='black',va='bottom',ha='center',fontsize=labelFontSize)

# plot and file

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')